In [1]:
import numpy as np
import pandas as pd

import pytwits
import requests

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [2]:
ticker_list = ["GOOG", "AMZN", "FB", "MSFT", "ADBE", "BABA", "BIDU", "BKNG", "BZUN", "MA", "MTCH", "MOMO", "NFLX", "PCTY", 
               "PYPL", "RP", "CRM", "SINA", "SQ", "ZG", "CTRP", "NVDA", "RNG", "WB", "ENV", "WUBA", "GDDY"]
tweets = []
tickers = []
access_token = '26c8e5b3d7bd2903f6091364ae85a91cce9b9aa4'
stocktwits = pytwits.StockTwits(access_token=access_token) # This would also work without passing token.

for ticker in ticker_list:
    symbol, cursor, messages = stocktwits.streams(path = 'symbol', id = ticker) # set limit on messages by limit = __, max = 30)
    for msg in messages:
        tweets.append(msg.body)
        tickers.append(symbol.symbol)    

tweet_df = pd.DataFrame({'Ticker': tickers, 'Tweet': tweets})

In [3]:
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("&#39;", "'")
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("$", "")
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace('&quot;', '"')
tweet_df['Tweet'] = tweet_df['Tweet'].str.replace("&amp;", "&")

In [4]:
def sentiment(tweet):
    analyze = SentimentIntensityAnalyzer()
    comp = analyze.polarity_scores(tweet)['compound']
    if comp > 0:
        return "Positive"
    elif comp < 0:
        return "Negative"
    else:
        return "Neutral"

def polarity(tweet):
    analyze = SentimentIntensityAnalyzer()
    return analyze.polarity_scores(tweet)['compound']

In [5]:
tweet_df['Sentiment'] = tweet_df['Tweet'].apply(lambda x: sentiment(x))
tweet_df['Score'] = tweet_df['Tweet'].apply(lambda x: polarity(x))
tweet_df

,Ticker,Tweet,Sentiment,Score
0,GOOG,@14index need data? JV with GOOG GOOGL !,Neutral,0.0000
1,GOOG,FB why the fuck has tech sold off so harshly w...,Negative,-0.8214
2,GOOG,GOOGL GOOG Look out below theres over 100 doll...,Neutral,0.0000
3,GOOG,"ES_F SPY SPX dips still getting bought, bears ...",Neutral,0.0000
4,GOOG,GOOG EXP:9/14/2018|MaxPain:1180.0|HighPutOI:11...,Neutral,0.0000
5,GOOG,AMRN SSC AMZN GOOG ABBV and last but not least...,Negative,-0.7299
6,GOOG,RUT SPY NDX DJIA EEM FB AAPL AMZN GOOG MSFT NV...,Positive,0.3818
7,GOOG,GOOGL FB GOOG EBAY - European Parliament back...,Positive,0.1280
8,GOOG,CGC MTBC AAPL AMZN AMD HMNY CRON TLRY GOOG V...,Neutral,0.0000
9,GOOG,Big Tech Companies to Appear Before Senate to ...,Neutral,0.0000


In [6]:
tweet_summary = tweet_df.groupby(['Ticker', 'Sentiment'], as_index=False).agg({'Tweet':'count','Score':'mean'}).rename(columns = {
    'Tweet':'Tweet_count', 'Score':'Avg_Score'})

In [7]:
tweet_summary

,Ticker,Sentiment,Tweet_count,Avg_Score
0,ADBE,Negative,6,-0.253733
1,ADBE,Neutral,17,0.000000
2,ADBE,Positive,7,0.479300
3,AMZN,Negative,8,-0.403875
4,AMZN,Neutral,12,0.000000
5,AMZN,Positive,10,0.487280
6,BABA,Negative,8,-0.437225
7,BABA,Neutral,10,0.000000
8,BABA,Positive,12,0.466433
9,BIDU,Negative,4,-0.611200


In [8]:
tweet_summary[(tweet_summary['Ticker'] == 'ZG') & (tweet_summary['Sentiment'] == 'Positive')].Tweet_count

80    4
Name: Tweet_count, dtype: int64